In [16]:
import joblib
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
from IPython.core.pylabtools import figsize
from IPython.display import IFrame
from pandas.api.types import CategoricalDtype
from scipy.sparse import csr_matrix
from sklearn import datasets
from sklearn import metrics
from sklearn import neighbors                           
from sklearn import tree, linear_model,neighbors   
from mlxtend.frequent_patterns import apriori, association_rules

In [2]:
#for product portforlio, we only focus on the sales data. 
#We group the sales data by customer, date and EquipmentCategory and regard the purchase of one customer at the same date as one purchase history. 

sales_with_category = pd.read_csv("sales_with_category.csv")
sales_with_category.info()
sales_category_list_cust_date = sales_with_category.groupby(['CustomerName','Sale Date','Base.Implement','EquipmentCategoryDesc']).size()
sales_category_list_cust_date

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (70,77,80,84) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55377 entries, 0 to 55376
Columns: 107 entries, Unnamed: 0 to sold_to_code_y
dtypes: float64(20), int64(2), object(85)
memory usage: 45.2+ MB


CustomerName                         Sale Date   Base.Implement  EquipmentCategoryDesc
#NAME? ROBERT FITCH                  05/20/2019  Base            AG Tractors              2
                                                 Implement       Material Handling        1
(MICKA) HARVEY CINDY                 03/24/2020  Base            Mowers                   1
- FIRST BAPTIST WYOMING              03/20/2019  Base            Mowers                   1
. KUTTING EDGE PROPERTY SERVICE LLC  09/25/2019  Base            Mowers                   1
                                                                                         ..
yovich matt                          04/13/2021  Base            Utility Vehicles         1
z915e                                06/18/2021  Base            Mowers                   1
zakostelecky chris                   10/29/2020  Base            Skid Steers              1
zimmerman                            06/20/2019  Base            Utility Vehicles    

In [3]:
# In this chunk, we unstack the data to meet the 
cust_purchase_history_date = sales_category_list_cust_date.to_frame().unstack().unstack()
cust_purchase_history_date.info()
cust_purchase_history_date
cust_purchase_history_date = sales_category_list_cust_date.to_frame().unstack().unstack()
cust_purchase_history_date1 = cust_purchase_history_date.dropna(axis = 1,how = 'all').sort_index(level = 2,axis = 1)
cust_purchase_history_date1.info()
cust_purchase_history_date1
cust_purchase_history_date1.to_csv('associate_rule_with_date&category.csv')

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 31415 entries, ('#NAME? ROBERT FITCH', '05/20/2019') to ('zimmerman', '06/20/2019')
Data columns (total 82 columns):
 #   Column                                                                     Non-Null Count  Dtype  
---  ------                                                                     --------------  -----  
 0   (0, AG Tractors, Base)                                                     5641 non-null   float64
 1   (0, AG Tractors, Implement)                                                0 non-null      float64
 2   (0, Material Handling, Base)                                               0 non-null      float64
 3   (0, Material Handling, Implement)                                          2067 non-null   float64
 4   (0, Mowers, Base)                                                          14755 non-null  float64
 5   (0, Mowers, Implement)                                                     0 non-null      float64
 

0            \
EquipmentCategoryDesc                          AG Tractors             
Base.Implement                                        Base Implement   
CustomerName                        Sale Date                          
#NAME? ROBERT FITCH                 05/20/2019         2.0       NaN   
(MICKA) HARVEY CINDY                03/24/2020         NaN       NaN   
- FIRST BAPTIST WYOMING             03/20/2019         NaN       NaN   
. KUTTING EDGE PROPERTY SERVICE LLC 09/25/2019         NaN       NaN   
0                                   03/24/2020         1.0       NaN   
...                                                    ...       ...   
yovich matt                         04/13/2021         NaN       NaN   
z915e                               06/18/2021         NaN       NaN   
zakostelecky chris                  10/29/2020         NaN       NaN   
zimmerman                           08/12/2019         NaN       NaN   
                                    06/20/2019         NaN       NaN   

                                                                            \
EquipmentCategoryDesc                          Material Handling             
Base.Implement                                              Base Implement   
CustomerName                        Sale Date                                
#NAME? ROBERT FITCH                 05/20/2019               NaN       1.0   
(MICKA) HARVEY CINDY                03/24/2020               NaN       NaN   
- FIRST BAPTIST WYOMING             03/20/2019               NaN       NaN   
. KUTTING EDGE PROPERTY SERVICE LLC 09/25/2019               NaN       NaN   
0                                   03/24/2020               NaN       NaN   
...                                                          ...       ...   
yovich matt                         04/13/2021               NaN       NaN   
z915e                               06/18/2021               NaN       NaN   
zakostelecky chris                  10/29/2020               NaN       NaN   
zimmerman                           08/12/2019               NaN       NaN   
                                    06/20/2019               NaN       NaN   

                                                                              \
EquipmentCategoryDesc                          Mowers           Hay & Forage   
Base.Implement                                   Base Implement         Base   
CustomerName                        Sale Date                                  
#NAME? ROBERT FITCH                 05/20/2019    NaN       NaN          NaN   
(MICKA) HARVEY CINDY                03/24/2020    1.0       NaN          NaN   
- FIRST BAPTIST WYOMING             03/20/2019    1.0       NaN          NaN   
. KUTTING EDGE PROPERTY SERVICE LLC 09/25/2019    1.0       NaN          NaN   
0                                   03/24/2020    NaN       NaN          NaN   
...                                               ...       ...          ...   
yovich matt                         04/13/2021    NaN       NaN          NaN   
z915e                               06/18/2021    1.0       NaN          NaN   
zakostelecky chris                  10/29/2020    NaN       NaN          NaN   
zimmerman                           08/12/2019    NaN       NaN          NaN   
                                    06/20/2019    NaN       NaN          NaN   

                                                                         \
EquipmentCategoryDesc                                    Snow Equipment   
Base.Implement                                 Implement           Base   
CustomerName                        Sale Date                             
#NAME? ROBERT FITCH                 05/20/2019       NaN            NaN   
(MICKA) HARVEY CINDY                03/24/2020       NaN            NaN   
- FIRST BAPTIST WYOMING             03/20/2019       NaN            NaN   
. KUTTING EDGE PROPERTY SERVICE LLC 09/25/2019       NaN            NaN   
0   

In [17]:
# I have save the prepared data.We could skip the former step and start from here.
cust_purchase_history_date2 = pd.read_csv('associate_rule_with_date&category.csv',index_col=0)
associate_rule_cust_date = cust_purchase_history_date2.iloc[1:,0:]
cust_purchase_history_date3 = cust_purchase_history_date2.iloc[1:,0:]
associate_rule_cust_date

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,4WD Loaders,AG Tractors,Combine Harvesting,Commercial Mowing,Compact 4WD Loaders,Compact Excavators,Compact Track Loaders,Crawlers,Excavators,Golf Products,...,Construction Tools,Cutters & Shredders,Landscape Equipment,Material Handling,Snow Equipment,Tractor Implements - Compact Utility and Utility Tractors,Trailers,Turf Attachments,Turf implements,Worksite Pro Implements
Sale Date,,,,,,,,,,,,,,,,,,,,,
5/20/19,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
3/24/20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3/20/19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9/25/19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3/24/20,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4/13/21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6/18/21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10/29/20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# Turning the data into binary format to meet the requirement of Apriori Algorithm.
for i in range(cust_purchase_history_date3.shape[0]):
    for j in range(cust_purchase_history_date3.shape[1]):
        if pd.isna(cust_purchase_history_date3.iloc[i][j]):
            associate_rule_cust_date.iloc[i][j] = 0
        else:
            associate_rule_cust_date.iloc[i][j] = 1

In [19]:
associate_rule_cust_date.info()

<class 'pandas.core.frame.DataFrame'>
Index: 31415 entries, 5/20/19 to 6/20/19
Data columns (total 41 columns):
 #   Column                                                     Non-Null Count  Dtype 
---  ------                                                     --------------  ----- 
 0   4WD Loaders                                                31415 non-null  object
 1   AG Tractors                                                31415 non-null  object
 2   Combine Harvesting                                         31415 non-null  object
 3   Commercial Mowing                                          31415 non-null  object
 4   Compact 4WD Loaders                                        31415 non-null  object
 5   Compact Excavators                                         31415 non-null  object
 6   Compact Track Loaders                                      31415 non-null  object
 7   Crawlers                                                   31415 non-null  object
 8   Excavators   

In [20]:
#Adding 2 columns names 'Base' and 'Implement' to prepare for next step.
new_df_cust_date = associate_rule_cust_date
new_df_cust_date['Base'] = 0
new_df_cust_date['Implement'] = 0
new_df_cust_date = new_df_cust_date.astype('float') 
new_df_cust_date.info()

<class 'pandas.core.frame.DataFrame'>
Index: 31415 entries, 5/20/19 to 6/20/19
Data columns (total 43 columns):
 #   Column                                                     Non-Null Count  Dtype 
---  ------                                                     --------------  ----- 
 0   4WD Loaders                                                31415 non-null  object
 1   AG Tractors                                                31415 non-null  object
 2   Combine Harvesting                                         31415 non-null  object
 3   Commercial Mowing                                          31415 non-null  object
 4   Compact 4WD Loaders                                        31415 non-null  object
 5   Compact Excavators                                         31415 non-null  object
 6   Compact Track Loaders                                      31415 non-null  object
 7   Crawlers                                                   31415 non-null  object
 8   Excavators   

In [21]:
new_df_cust_date

,4WD Loaders,AG Tractors,Combine Harvesting,Commercial Mowing,Compact 4WD Loaders,Compact Excavators,Compact Track Loaders,Crawlers,Excavators,Golf Products,...,Landscape Equipment,Material Handling,Snow Equipment,Tractor Implements - Compact Utility and Utility Tractors,Trailers,Turf Attachments,Turf implements,Worksite Pro Implements,Base,Implement
Sale Date,,,,,,,,,,,,,,,,,,,,,
5/20/19,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3/24/20,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3/20/19,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9/25/19,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3/24/20,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4/13/21,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6/18/21,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10/29/20,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
# The purpose of this step is that to check if this customer purchase the base machine or implemnts.

m = []
for i in range(new_df_cust_date.shape[0]):
    if (sum(new_df_cust_date.iloc[i,:26]) == 0):
        m.append(0)
    else:
        m.append(1)
n = []
for i in range(new_df_cust_date.shape[0]):
    if (sum(new_df_cust_date.iloc[i,26:]) == 0):
        n.append(0)
    else:
        n.append(1)
    
new_df_cust_date['Base'] = m
new_df_cust_date['Implement'] = n
new_df_cust_date.info()

<class 'pandas.core.frame.DataFrame'>
Index: 31415 entries, 5/20/19 to 6/20/19
Data columns (total 43 columns):
 #   Column                                                     Non-Null Count  Dtype 
---  ------                                                     --------------  ----- 
 0   4WD Loaders                                                31415 non-null  object
 1   AG Tractors                                                31415 non-null  object
 2   Combine Harvesting                                         31415 non-null  object
 3   Commercial Mowing                                          31415 non-null  object
 4   Compact 4WD Loaders                                        31415 non-null  object
 5   Compact Excavators                                         31415 non-null  object
 6   Compact Track Loaders                                      31415 non-null  object
 7   Crawlers                                                   31415 non-null  object
 8   Excavators   

In [23]:
new_df_cust_date

,4WD Loaders,AG Tractors,Combine Harvesting,Commercial Mowing,Compact 4WD Loaders,Compact Excavators,Compact Track Loaders,Crawlers,Excavators,Golf Products,...,Landscape Equipment,Material Handling,Snow Equipment,Tractor Implements - Compact Utility and Utility Tractors,Trailers,Turf Attachments,Turf implements,Worksite Pro Implements,Base,Implement
Sale Date,,,,,,,,,,,,,,,,,,,,,
5/20/19,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,1
3/24/20,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3/20/19,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
9/25/19,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3/24/20,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4/13/21,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
6/18/21,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
10/29/20,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [24]:
new_df_cust_date.index = pd.to_datetime(new_df_cust_date.index)

In [25]:
#new_df_cust_date

,4WD Loaders,AG Tractors,Combine Harvesting,Commercial Mowing,Compact 4WD Loaders,Compact Excavators,Compact Track Loaders,Crawlers,Excavators,Golf Products,...,Landscape Equipment,Material Handling,Snow Equipment,Tractor Implements - Compact Utility and Utility Tractors,Trailers,Turf Attachments,Turf implements,Worksite Pro Implements,Base,Implement
Sale Date,,,,,,,,,,,,,,,,,,,,,
2019-05-20,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,1
2020-03-24,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2019-03-20,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2019-09-25,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2020-03-24,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-13,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2021-06-18,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2020-10-29,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [26]:
Jan1 = new_df_cust_date.loc['2019-01']
Jan2 = new_df_cust_date.loc['2020-01']
Jan3 = new_df_cust_date.loc['2021-01']
Jan = pd.concat([Jan1,Jan2,Jan3],axis=0)


Feb1 = new_df_cust_date.loc['2019-02']
Feb2 = new_df_cust_date.loc['2020-02']
Feb3 = new_df_cust_date.loc['2021-02']
Feb = pd.concat([Feb1,Feb2,Feb3],axis=0)

Mar1 = new_df_cust_date.loc['2019-03']
Mar2 = new_df_cust_date.loc['2020-03']
Mar3 = new_df_cust_date.loc['2021-03']
Mar = pd.concat([Mar1,Mar2,Mar3],axis=0)

Apr1 = new_df_cust_date.loc['2019-04']
Apr2 = new_df_cust_date.loc['2020-04']
Apr3 = new_df_cust_date.loc['2021-04']
Apr = pd.concat([Apr1,Apr2,Apr3],axis=0)


May1 = new_df_cust_date.loc['2019-05']
May2 = new_df_cust_date.loc['2020-05']
May3 = new_df_cust_date.loc['2021-05']
May = pd.concat([May1,May2,May3],axis=0)

Jun1 = new_df_cust_date.loc['2019-06']
Jun2 = new_df_cust_date.loc['2020-06']
Jun3 = new_df_cust_date.loc['2021-06']
Jun = pd.concat([Jun1,Jun2,Jun3],axis=0)

Jul1 = new_df_cust_date.loc['2019-07']
Jul2 = new_df_cust_date.loc['2020-07']
Jul3 = new_df_cust_date.loc['2021-07']
Jul = pd.concat([Jul1,Jul2,Jul3],axis=0)

Aug1 = new_df_cust_date.loc['2019-08']
Aug2 = new_df_cust_date.loc['2020-08']
Aug3 = new_df_cust_date.loc['2021-08']
Aug = pd.concat([Aug1,Aug2,Aug3],axis=0)

Sep1 = new_df_cust_date.loc['2019-09']
Sep2 = new_df_cust_date.loc['2020-09']
Sep3 = new_df_cust_date.loc['2021-09']
Sep = pd.concat([Sep1,Sep2,Sep3],axis=0)

Oct1 = new_df_cust_date.loc['2019-10']
Oct2 = new_df_cust_date.loc['2020-10']
Oct3 = new_df_cust_date.loc['2021-10']
Oct = pd.concat([Oct1,Oct2,Oct3],axis=0)

Nov1 = new_df_cust_date.loc['2019-11']
Nov2 = new_df_cust_date.loc['2020-11']
Nov3 = new_df_cust_date.loc['2021-11']
Nov = pd.concat([Nov1,Nov2,Nov3],axis=0)

Dec1 = new_df_cust_date.loc['2019-12']
Dec2 = new_df_cust_date.loc['2020-12']
Dec3 = new_df_cust_date.loc['2021-12']
Dec = pd.concat([Dec1,Dec2,Dec3],axis=0)

In [27]:
spring = pd.concat([Mar,Apr,May],axis=0)
summer = pd.concat([Jun,Jul,Aug],axis=0)
fall = pd.concat([Sep,Oct,Nov],axis=0)
winter = pd.concat([Dec,Jan,Feb],axis=0)

In [104]:
Jan_baseonly = Jan.iloc[0:,26:42]
support_Jan_baseonly = apriori(Jan_baseonly, min_support = 0.01, use_colnames = True, verbose = 1)
support_Jan_baseonly
support_Jan = apriori(Jan.iloc[0:,:41], min_support = 0.005, use_colnames = True, verbose = 1)
support_Jan
association_Jan = association_rules(support_Jan, metric = "confidence", min_threshold = 0.1)
association_Jan


Processing 42 combinations | Sampling itemset size 2

Processing 600 combinations | Sampling itemset size 2
Processing 51 combinations | Sampling itemset size 3


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Agriculture Implements),(AG Tractors),0.042418,0.270414,0.018028,0.425000,1.571667,0.006557,1.268846
1,(Cutters & Shredders),(AG Tractors),0.023330,0.270414,0.007423,0.318182,1.176649,0.001114,1.070060
2,(Landscape Equipment),(AG Tractors),0.040297,0.270414,0.009544,0.236842,0.875851,-0.001353,0.956010
3,(AG Tractors),(Material Handling),0.270414,0.103924,0.028632,0.105882,1.018848,0.000530,1.002191
4,(Material Handling),(AG Tractors),0.103924,0.270414,0.028632,0.275510,1.018848,0.000530,1.007035
5,(Snow Equipment),(AG Tractors),0.125133,0.270414,0.014846,0.118644,0.438750,-0.018991,0.827800
6,(Turf implements),(AG Tractors),0.101803,0.270414,0.018028,0.177083,0.654861,-0.009501,0.886586
7,(Snow Equipment),(Mowers),0.125133,0.148462,0.012725,0.101695,0.684988,-0.005852,0.947938
8,(Turf implements),(Mowers),0.101803,0.148462,0.012725,0.125000,0.841964,-0.002389,0.973186
9,(Agriculture Implements),(Snow Equipment),0.042418,0.125133,0.005302,0.125000,0.998941,-0.000006,0.999849


In [108]:
summer_baseonly = summer.iloc[0:,26:42]
support_summer_baseonly = apriori(summer_baseonly, min_support = 0.01, use_colnames = True, verbose = 1)
#support_summer_baseonly.to_csv('support_summer_baseonly.csv')
support_summer = apriori(summer.iloc[0:,:41], min_support = 0.005, use_colnames = True, verbose = 1)
support_summer
#support_summer.to_csv('support_summer.csv')
association_summer = association_rules(support_summer, metric = "confidence", min_threshold = 0.1)
association_summer
#association_summer.to_csv('association_summer.csv')


Processing 42 combinations | Sampling itemset size 2

Processing 210 combinations | Sampling itemset size 2
Processing 66 combinations | Sampling itemset size 3


In [109]:
winter_baseonly = winter.iloc[0:,26:42]
support_winter_baseonly = apriori(winter_baseonly, min_support = 0.01, use_colnames = True, verbose = 1)
support_winter_baseonly
#support_winter_baseonly.to_csv('support_winter_baseonly.csv')
support_winter = apriori(winter.iloc[0:,:41], min_support = 0.005, use_colnames = True, verbose = 1)
support_winter
#support_winter.to_csv('support_winter.csv')
association_winter = association_rules(support_winter, metric = "confidence", min_threshold = 0.1)
association_winter
#association_winter.to_csv('association_winter.csv')


Processing 42 combinations | Sampling itemset size 2

Processing 506 combinations | Sampling itemset size 2
Processing 66 combinations | Sampling itemset size 3
